# Incremental Training with MLJFlux

This demonstration is available as a Jupyter notebook or julia script
[here](https://github.com/FluxML/MLJFlux.jl/tree/dev/docs/src/common_workflows/incremental_training).

In this workflow example we explore how to incrementally train MLJFlux models.

In [1]:
using Pkg
PKG_ENV = joinpath(@__DIR__, "..", "..", "..")
Pkg.activate(PKG_ENV);
Pkg.instantiate();

  Activating project at `~/GoogleDrive/Julia/MLJ/MLJFlux/docs`


**Julia version** is assumed to be 1.10.*

### Basic Imports

In [2]:
using MLJ               # Has MLJFlux models
using Flux              # For more flexibility
import Optimisers       # native Flux.jl optimisers no longer supported
using StableRNGs        # for reproducibility across Julia versions

stable_rng() = StableRNGs.StableRNG(123)

stable_rng (generic function with 1 method)

### Loading and Splitting the Data

In [3]:
iris = load_iris() # a named-tuple of vectors
y, X = unpack(iris, ==(:target), rng=stable_rng())
X = fmap(column-> Float32.(column), X) # Flux prefers Float32 data
(X_train, X_test), (y_train, y_test) = partition(
    (X, y), 0.8,
    multi = true,
    shuffle = true,
    rng=stable_rng(),
);

### Instantiating the model

Now let's construct our model. This follows a similar setup to the one followed in the
[Quick Start](../../index.md#Quick-Start).

In [4]:
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg=MLJFlux
clf = NeuralNetworkClassifier(
    builder=MLJFlux.MLP(; hidden=(5,4), σ=Flux.relu),
    optimiser=Optimisers.Adam(0.01),
    batch_size=8,
    epochs=10,
    rng=stable_rng(),
)

[ Info: For silent loading, specify `verbosity=0`. 
import MLJFlux ✔


NeuralNetworkClassifier(
  builder = MLP(
        hidden = (5, 4), 
        σ = NNlib.relu), 
  finaliser = NNlib.softmax, 
  optimiser = Optimisers.Adam(eta=0.01, beta=(0.9, 0.999), epsilon=1.0e-8), 
  loss = Flux.Losses.crossentropy, 
  epochs = 10, 
  batch_size = 8, 
  lambda = 0.0, 
  alpha = 0.0, 
  rng = StableRNGs.LehmerRNG(state=0x000000000000000000000000000000f7), 
  optimiser_changes_trigger_retraining = false, 
  acceleration = ComputationalResources.CPU1{Nothing}(nothing), 
  embedding_dims = Dict{Symbol, Real}())

### Initial round of training

Now let's train the model. Calling fit! will automatically train it for 100 epochs as
specified above.

In [5]:
mach = machine(clf, X_train, y_train)
fit!(mach, verbosity=0)

trained Machine; caches model-specific representations of data
  model: NeuralNetworkClassifier(builder = MLP(hidden = (5, 4), …), …)
  args: 
    1:	Source @040 ⏎ ScientificTypesBase.Table{AbstractVector{ScientificTypesBase.Continuous}}
    2:	Source @771 ⏎ AbstractVector{ScientificTypesBase.Multiclass{3}}


Let's evaluate the training loss and validation accuracy

In [6]:
training_loss = cross_entropy(predict(mach, X_train), y_train)

0.22537291610526708

In [7]:
val_acc = accuracy(predict_mode(mach, X_test), y_test)

0.9333333333333333

Poor performance it seems.

### Incremental Training

Now let's train it for another 30 epochs at half the original learning rate. All we need
to do is changes these hyperparameters and call fit again. It won't reset the model
parameters before training.

In [8]:
clf.optimiser = Optimisers.Adam(clf.optimiser.eta/2)
clf.epochs = clf.epochs + 30
fit!(mach, verbosity=2);

[ Info: Updating machine(NeuralNetworkClassifier(builder = MLP(hidden = (5, 4), …), …), …).
[ Info: Loss is 0.2124
[ Info: Loss is 0.1803
[ Info: Loss is 0.1545
[ Info: Loss is 0.1345
[ Info: Loss is 0.1192
[ Info: Loss is 0.1076
[ Info: Loss is 0.09877
[ Info: Loss is 0.09212
[ Info: Loss is 0.08716
[ Info: Loss is 0.08347
[ Info: Loss is 0.08059
[ Info: Loss is 0.07807
[ Info: Loss is 0.07564
[ Info: Loss is 0.07322
[ Info: Loss is 0.07085
[ Info: Loss is 0.06863
[ Info: Loss is 0.06663
[ Info: Loss is 0.06484
[ Info: Loss is 0.06322
[ Info: Loss is 0.06174
[ Info: Loss is 0.06039
[ Info: Loss is 0.05912
[ Info: Loss is 0.05794
[ Info: Loss is 0.05687
[ Info: Loss is 0.05586
[ Info: Loss is 0.05491
[ Info: Loss is 0.05402
[ Info: Loss is 0.05317
[ Info: Loss is 0.05238
[ Info: Loss is 0.05162


Let's evaluate the training loss and validation accuracy

In [9]:
training_loss = cross_entropy(predict(mach, X_train), y_train)

0.04072631383146386

In [10]:
training_acc = accuracy(predict_mode(mach, X_test), y_test)

0.9333333333333333

That's much better. If we are rather interested in resetting the model parameters before
fitting, we can do `fit(mach, force=true)`.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*